In [ ]:
%matplotlib qt
# Używamy %matplotlib qt (lub %matplotlib notebook) 
# aby animacja działała poprawnie w środowiskach typu Jupyter/IPython.

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import time
import random

# --- Krok 1: Definicje i Ustawienia (Bez zmian) ---
# (Wszystkie definicje WORLD, BATTLEFIELD_UNITS, SYSTEM_TYPES, 
#  DRONE_TYPES, ROUTES, SPAWN_WAVE, SIMULATION_SETTINGS
#  pozostają takie same jak w poprzedniej wersji)

WORLD_SETTINGS = {
    'width': 1000,
    'height': 1000
}

BATTLEFIELD_UNITS = [
    {"id": "cram_1", "type": "C-RAM", "position": (400, 500)},
    {"id": "cram_2", "type": "C-RAM", "position": (400, 700)},
    {"id": "laser_1", "type": "LASER", "position": (600, 600)}
]

SYSTEM_TYPES = {
    "C-RAM": {
        "range": 250, "channels": 1, "service_time": 0.5,
        "damage_per_hit": 1, "magazine_size": 150, "reload_time": 6.0,
        "color": 'blue'
    },
    "LASER": {
        "range": 400, "channels": 2, "service_time": 3.0,
        "damage_per_hit": 3, "magazine_size": 4, "reload_time": 8.0,
        "color": 'red'
    }
}

DRONE_TYPES = {
    "Scout": {
        "speed": 60, "hp": 1, "sway_strength": 20.0, "color": 'gray'
    },
    "Bomber": {
        "speed": 35, "hp": 3, "sway_strength": 10.0, "color": 'black'
    }
}

ROUTES = {
    "Route_North": { "start": (500, 1000), "target": (500, 0) },
    "Route_West": { "start": (0, 700), "target": (1000, 700) },
    "Route_East": { "start": (1000, 300), "target": (0, 300) }
}

SPAWN_WAVE = [
    {"route": "Route_North", "type": "Bomber", "count": 8, "interval": 3.0},
    {"route": "Route_West", "type": "Scout", "count": 5, "interval": 2.0},
    {"route": "Route_East", "type": "Scout", "count": 5, "interval": 2.5},
    {"route": "Route_North", "type": "Bomber", "count": 5, "interval": 1.0}
]

SIMULATION_SETTINGS = {
    "dt": 0.1,
    "targeting_strategy": 'NEAREST',
    "is_coordinated": True
}


# --- Krok 2: Definicje Klas (Obiekty) ---

def print_log(sim_time, message):
    print(f"[{sim_time:6.1f}s] {message}")

# --- Klasa Drone (Bez zmian) ---
class Drone:
    def __init__(self, route_name, route_info, drone_type_name):
        self.x, self.y = route_info['start']
        self.target_x, self.target_y = route_info['target']
        self.route_name = route_name
        self.type_name = drone_type_name
        self.props = DRONE_TYPES[drone_type_name]
        self.speed = self.props['speed']
        self.color = self.props['color']
        self.hp = self.props['hp']
        self.sway_strength = self.props['sway_strength']
        self.status = 'active'
        self.time_in_world = 0.0
        self.vx = 0.0
        self.vy = 0.0

    def move(self, dt):
        if self.status != 'active': return
        dx_target = self.target_x - self.x
        dy_target = self.target_y - self.y
        dist_to_target = np.hypot(dx_target, dy_target)
        if self.check_status(dist_to_target, dt): return
        vx_base = (dx_target / dist_to_target) * self.speed
        vy_base = (dy_target / dist_to_target) * self.speed
        if self.speed > 0:
            perp_vx_norm = -vy_base / self.speed
            perp_vy_norm = vx_base / self.speed
        else:
            perp_vx_norm, perp_vy_norm = 0, 0
        sway_force = self.sway_strength * random.uniform(-1, 1)
        self.vx = vx_base + perp_vx_norm * sway_force
        self.vy = vy_base + perp_vy_norm * sway_force
        self.x += self.vx * dt
        self.y += self.vy * dt
        self.time_in_world += dt
        
    def check_status(self, dist_to_target, dt):
        if dist_to_target < (self.speed * dt * 1.5):
            self.status = 'leaked'
            return True
        return False

    def take_hit(self, damage, sim_time):
        if self.status != 'active': return
        self.hp -= damage
        if self.hp <= 0:
            self.status = 'destroyed'
            print_log(sim_time, f"Dron {self.type_name} zniszczony!")
            
    def get_distance_to_goal(self):
        return np.hypot(self.target_x - self.x, self.target_y - self.y)

# --- Klasa DefenseSystem (Gruntownie przebudowana) ---
class DefenseSystem:
    """Reprezentuje system obronny (serwer). Zarządza swoim stanem."""
    def __init__(self, unit_info, system_types_dict):
        self.id = unit_info['id']
        self.x, self.y = unit_info['position']
        self.type = unit_info['type']
        self.props = system_types_dict[self.type]
        self.range = self.props['range']
        self.channels = self.props['channels']
        self.service_time = self.props['service_time']
        self.damage = self.props['damage_per_hit']
        self.magazine_size = self.props['magazine_size']
        self.reload_time = self.props['reload_time']
        self.color = self.props['color']
        self.status = 'IDLE'
        self.current_ammo = self.magazine_size
        self.reload_timer = 0.0
        self.targets = {}
        
        # Statystyki do wykresów
        self.total_time_busy = 0.0
        self.total_time_reloading = 0.0
        self.targets_destroyed = 0

    def get_free_channels(self):
        return self.channels - len(self.targets)
    def can_engage(self):
        # System może namierzać, nawet jeśli ma 0 amunicji,
        # ale strzał zostanie zablokowany.
        return self.status != 'RELOADING'
    def is_targeting(self, drone):
        return drone in self.targets

    def update_state(self, dt, sim_time):
        """Aktualizuje wewnętrzne timery (przeładowanie, obsługa celu)."""
        
        # 1. Logika przeładowania
        if self.status == 'RELOADING':
            self.reload_timer -= dt
            self.total_time_reloading += dt
            if self.reload_timer <= 0:
                self.status = 'IDLE'
                self.current_ammo = self.magazine_size
                print_log(sim_time, f"System {self.id} zakończył przeładowanie (Amunicja: {self.current_ammo})")
            return # Podczas przeładowania nie można robić NIC innego
        
        # 2. Logika obsługi celów (jeśli nie przeładowuje)
        if not self.targets:
            self.status = 'IDLE' # Nie ma celów, wróć do IDLE
            return

        # Jeśli tu dotarliśmy, mamy cele, więc jesteśmy 'BUSY'
        self.status = 'BUSY'
        self.total_time_busy += dt
            
        for drone, timer in list(self.targets.items()):
            
            # A. Sprawdź, czy cel jest wciąż ważny (w zasięgu i aktywny)
            dist = np.hypot(drone.x - self.x, drone.y - self.y)
            if drone.status != 'active' or dist > self.range:
                self._remove_target(drone)
                continue 
            
            # B. Cel jest ważny, aktualizuj timer "obsługi"
            timer -= dt
            
            # C. Jeśli timer <= 0 -> Próba STRZAŁU
            if timer <= 0:
                
                # C.1. Sprawdź amunicję
                if self.current_ammo > 0:
                    # Mamy amunicję -> STRZAŁ
                    self._fire_at_target(drone, sim_time)
                    
                    if drone.status == 'destroyed':
                        self._remove_target(drone)
                        self.targets_destroyed += 1
                    else:
                        # Reset timera na następny strzał (logika C-RAM)
                        self.targets[drone] = self.service_time 
                else:
                    # C.2. Brak amunicji -> "Klik" i przeładowanie
                    print_log(sim_time, f"System {self.id} [Klik] Brak amunicji!")
                    self._start_reloading(sim_time) # Rozpoczyna przeładowanie i CZYŚCI cele
                    break # Przerwij pętlę celów, bo system przeładowuje
            else:
                # D. Jeśli timer > 0, kontynuuj śledzenie
                self.targets[drone] = timer 

    def engage_target(self, drone, sim_time):
        """Rozpoczyna "obsługę" nowego celu. Wywoływane przez Symulację."""
        # Ta funkcja JUŻ NIE ZUŻYWA amunicji
        if not self.can_engage() or self.get_free_channels() <= 0:
            return # Nie można namierzyć
        
        self.targets[drone] = self.service_time
        print_log(sim_time, f"System {self.id} namierzył {drone.type_name}")
        # Nie zmieniamy amunicji, robimy to przy strzale

    def _fire_at_target(self, drone, sim_time):
        """NOWA FUNKCJA: Zadaje obrażenia ORAZ zużywa amunicję."""
        if self.current_ammo <= 0: return # Zabezpieczenie
            
        self.current_ammo -= 1
        print_log(sim_time, f"System {self.id} strzela do {drone.type_name}! (Amunicja: {self.current_ammo})")
        drone.take_hit(self.damage, sim_time)

        if self.current_ammo <= 0 and self.magazine_size > 0:
            print_log(sim_time, f"System {self.id} [Ostatni Strzał]... ")
            self._start_reloading(sim_time) # Przeładuj i wyczyść cele

    def _start_reloading(self, sim_time):
        """NOWA FUNKCJA: Centralizuje logikę przeładowania."""
        if self.status == 'RELOADING': return # Już przeładowuje
        
        print_log(sim_time, f"System {self.id} przeładowuje ({self.reload_time}s)")
        self.status = 'RELOADING'
        self.reload_timer = self.reload_time
        
        # --- KLUCZOWA POPRAWKA (dla Pana błędu) ---
        # Natychmiast "puść" wszystkie cele, bo system przeładowuje
        self.targets.clear() 
        # --- KONIEC POPRAWKI ---

    def _remove_target(self, drone):
        """Usuwa drona z listy aktywnych celów."""
        if drone in self.targets:
            del self.targets[drone]

    def get_distance_to_drone(self, drone):
        return np.hypot(drone.x - self.x, drone.y - self.y)

# --- Klasa Simulation (Bez zmian, oprócz wywołania) ---
class Simulation:
    def __init__(self, world, sim_settings, system_types, units, drone_types, routes, wave):
        self.world = world
        self.sim_settings = sim_settings
        self.system_types = system_types
        self.drone_types = drone_types
        self.routes = routes
        self.sim_time = 0.0
        self.dt = sim_settings['dt']
        self.active_drones = []
        self.leaked_drones = []
        self.destroyed_drones = []
        self.systems = [DefenseSystem(u, self.system_types) for u in units]
        self.spawn_plan = list(wave)
        self.next_spawn_time = 0.0
        self.spawn_interval = 0.0
        self.drones_to_spawn_in_batch = 0
        self.current_batch_info = {}
        self._setup_next_batch()

    def _setup_next_batch(self):
        if not self.spawn_plan:
            self.drones_to_spawn_in_batch = 0
            print_log(self.sim_time, "Wszystkie fale dronów zostały wysłane.")
            return
        self.current_batch_info = self.spawn_plan.pop(0)
        self.drones_to_spawn_in_batch = self.current_batch_info['count']
        self.spawn_interval = self.current_batch_info['interval']
        self.next_spawn_time = self.sim_time + self.spawn_interval
        msg = f"Rozpoczynanie fali: {self.current_batch_info['type']} " \
              f"na trasie {self.current_batch_info['route']} " \
              f"({self.drones_to_spawn_in_batch} szt. co {self.spawn_interval}s)"
        print_log(self.sim_time, msg)

    def _spawn_drone(self):
        route_name = self.current_batch_info['route']
        route_info = self.routes[route_name]
        drone_type = self.current_batch_info['type']
        new_drone = Drone(route_name, route_info, drone_type)
        self.active_drones.append(new_drone)
        self.drones_to_spawn_in_batch -= 1
        self.next_spawn_time += self.spawn_interval
        if self.drones_to_spawn_in_batch <= 0:
            self._setup_next_batch()

    def step(self):
        if self.drones_to_spawn_in_batch > 0 and self.sim_time >= self.next_spawn_time:
            self._spawn_drone()
        
        for drone in self.active_drones[:]:
            drone.move(self.dt)
            if drone.status == 'leaked':
                self.active_drones.remove(drone)
                self.leaked_drones.append(drone)
                print_log(self.sim_time, f"Dron {drone.type_name} PRZECIEKŁ na trasie {drone.route_name}.")
            elif drone.status == 'destroyed':
                self.active_drones.remove(drone)
                self.destroyed_drones.append(drone)
        
        for system in self.systems:
            system.update_state(self.dt, self.sim_time)
        
        self._assign_new_targets()
        self.sim_time += self.dt

    def _assign_new_targets(self):
        globally_engaged_drones = set()
        if self.sim_settings['is_coordinated']:
            for s in self.systems:
                globally_engaged_drones.update(s.targets.keys())
        for system in self.systems:
            free_channels = system.get_free_channels()
            if not system.can_engage() or free_channels <= 0:
                continue
            
            if system.magazine_size > 0:
                if system.current_ammo <= len(system.targets):
                    continue 

            potential_targets = []
            for drone in self.active_drones:
                dist = system.get_distance_to_drone(drone)
                if dist > system.range: continue
                if system.is_targeting(drone): continue
                if self.sim_settings['is_coordinated'] and drone in globally_engaged_drones:
                    continue
                potential_targets.append(drone)
            if not potential_targets: continue
            strategy = self.sim_settings['targeting_strategy']
            if strategy == 'NEAREST':
                potential_targets.sort(key=lambda d: system.get_distance_to_drone(d))
            elif strategy == 'OLDEST':
                potential_targets.sort(key=lambda d: d.time_in_world, reverse=True)
            elif strategy == 'NEAREST_GOAL':
                potential_targets.sort(key=lambda d: d.get_distance_to_goal())
            targets_to_assign = potential_targets[:free_channels]
            for target in targets_to_assign:
                system.engage_target(target, self.sim_time) # < Zmiana
                if self.sim_settings['is_coordinated']:
                    globally_engaged_drones.add(target)

    def is_finished(self):
        no_active = not self.active_drones
        no_more_spawns = self.drones_to_spawn_in_batch <= 0 and not self.spawn_plan
        return no_active and no_more_spawns

# --- Krok 3: Wizualizacja (Animacja) ---
# (Cała sekcja wizualizacji: fig, ax, sim, ani,
#  draw_static_environment, draw_systems, draw_drones,
#  update_plot, main - pozostają bez zmian)

fig, ax = plt.subplots(figsize=(10, 10))
sim = Simulation(WORLD_SETTINGS, SIMULATION_SETTINGS, SYSTEM_TYPES, 
                 BATTLEFIELD_UNITS, DRONE_TYPES, ROUTES, SPAWN_WAVE)
ani = None

def draw_static_environment(ax):
    ax.set_aspect('equal')
    ax.set_xlim(0, sim.world['width'])
    ax.set_ylim(0, sim.world['height'])
    ax.grid(True, linestyle=':', alpha=0.6)
    
    legend_handles = []
    for route_name, route_info in sim.routes.items():
        start, target = route_info['start'], route_info['target']
        ax.plot([start[0], target[0]], [start[1], target[1]], 'k:', alpha=0.3)
        h, = ax.plot([], [], 'X', color='black', markersize=15, alpha=0.5,
                     label=f"Cel: {route_name}")
        legend_handles.append(h)
        ax.plot(target[0], target[1], 'X', color='black', markersize=15, alpha=0.5)

    ax.legend(handles=legend_handles, loc='upper left', bbox_to_anchor=(0, 0.95))

def draw_systems(ax):
    for system in sim.systems:
        x, y = system.x, system.y
        status_color = 'gray'
        if system.status == 'BUSY': status_color = 'red'
        elif system.status == 'RELOADING': status_color = 'orange'
        
        zone = patches.Circle((x, y), system.range, 
                              facecolor=status_color, alpha=0.1, 
                              edgecolor=status_color, linestyle='--')
        ax.add_patch(zone)
        ax.plot(x, y, 'x', color=system.color, markersize=10, markeredgewidth=3)
        ammo_text = f"{system.current_ammo}" if system.magazine_size > 0 else "Inf"
        if system.status == 'RELOADING':
            ammo_text = f"RELOAD ({system.reload_timer:.1f}s)"
        ax.text(x + 10, y + 10, f"{system.id}\nAMMO: {ammo_text}", 
                fontsize=9, color='black', ha='left')
        
        # Ta linia jest TERAZ poprawna: system.targets jest czyszczone
        # przy przeładowaniu, więc linie znikną
        for drone in system.targets:
            ax.plot([x, drone.x], [y, drone.y], color='red', linestyle='-', linewidth=0.5)

def draw_drones(ax):
    for drone in sim.active_drones:
        ax.plot(drone.x, drone.y, 'o', color=drone.color, markersize=6)
        if drone.hp < DRONE_TYPES[drone.type_name]['hp']:
             ax.text(drone.x, drone.y - 10, f"HP: {drone.hp}", fontsize=8, color='red', ha='center')
    for drone in sim.destroyed_drones:
        ax.plot(drone.x, drone.y, 'x', color='red', markersize=5, alpha=0.7)
    for drone in sim.leaked_drones:
        ax.plot(drone.target_x, drone.target_y, 'v', color='gray', markersize=8, alpha=0.7)

def update_plot(frame):
    global ani, sim
    if not sim.is_finished():
        sim.step()
    
    ax.clear()
    draw_static_environment(ax)
    draw_systems(ax)
    draw_drones(ax)
    
    ax.set_title(f"Symulacja [Krok 4.3: Poprawiona Logika] | Czas: {sim.sim_time:.1f}s\n"
                 f"Zniszczone: {len(sim.destroyed_drones)} | "
                 f"Przeciekło: {len(sim.leaked_drones)} | "
                 f"Strategia: {sim.sim_settings['targeting_strategy']} (Koor: {sim.sim_settings['is_coordinated']})")
    ax.set_xlabel("Pozycja X (metry)")
    ax.set_ylabel("Pozycja Y (metry)")

    if sim.is_finished():
        print("--- Symulacja [Krok 4.3] zakończona ---")
        print(f"Całkowity czas: {sim.sim_time:.1f}s")
        print(f"Zniszczone: {len(sim.destroyed_drones)}")
        print(f"Przeciekło: {len(sim.leaked_drones)}")
        if ani:
            ani.event_source.stop()
        
        # --- KROK 5 (Zapowiedź) ---
        # plot_final_statistics(sim) 

def main():
    global ani, sim
    print("Uruchamianie animacji [Krok 4.3: Poprawiona Logika]...")
    
    sim = Simulation(WORLD_SETTINGS, SIMULATION_SETTINGS, SYSTEM_TYPES, 
                     BATTLEFIELD_UNITS, DRONE_TYPES, ROUTES, SPAWN_WAVE)
    
    ani = animation.FuncAnimation(fig, update_plot, 
                                  interval=(sim.dt * 1000), 
                                  blit=False)
    plt.show()

if __name__ == "__main__":
    main()

[   0.0s] Rozpoczynanie fali: Bomber na trasie Route_North (8 szt. co 3.0s)
Uruchamianie animacji [Krok 4.3: Poprawiona Logika]...
[   0.0s] Rozpoczynanie fali: Bomber na trasie Route_North (8 szt. co 3.0s)


C:\Users\mrmro\AppData\Local\Temp\ipykernel_18512\305022316.py:463: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, update_plot,


[   3.3s] System laser_1 namierzył Bomber
[   6.3s] System laser_1 strzela do Bomber! (Amunicja: 3)
[   6.3s] Dron Bomber zniszczony!
[   6.3s] System cram_2 namierzył Bomber
[   6.4s] System laser_1 namierzył Bomber
[   9.4s] System laser_1 strzela do Bomber! (Amunicja: 2)
[   9.4s] Dron Bomber zniszczony!
[   9.4s] System cram_2 namierzył Bomber
[   9.4s] System laser_1 namierzył Bomber
[  12.4s] System laser_1 strzela do Bomber! (Amunicja: 1)
[  12.4s] Dron Bomber zniszczony!
[  12.4s] System cram_2 namierzył Bomber
[  12.4s] System laser_1 namierzył Bomber
[  15.4s] System laser_1 strzela do Bomber! (Amunicja: 0)
[  15.4s] Dron Bomber zniszczony!
[  15.4s] System laser_1 [Ostatni Strzał]... 
[  15.4s] System laser_1 przeładowuje (8.0s)
[  15.4s] System cram_2 namierzył Bomber
[  17.1s] System cram_2 namierzył Bomber
[  17.7s] System cram_2 strzela do Bomber! (Amunicja: 149)
[  18.3s] System cram_2 strzela do Bomber! (Amunicja: 148)
[  18.9s] System cram_2 strzela do Bomber! (Amunic